# Quantum Computing on Azure Quantum with Q# and Jupyter Notebook: <br> Parallel QRNG

This notebook demonstrates how to execute a Q# program on Azure Quantum.

## Define Q# operations

The quickest way to define a Q# operation in a Q# Jupyter Notebook is to simply write the Q# code directly into a notebook cell and execute it. Note that for convenience, the `Microsoft.Quantum.Canon` and `Microsoft.Quantum.Intrinsic` namespaces are automatically opened in every cell.

In [1]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;
    
operation SampleRandomNumber(nQubits : Int) : Result[] {
    // We prepare a register of qubits in a uniform
    // superposition state, such that when we measure,
    // all bitstrings occur with equal probability.
    use register = Qubit[nQubits] {
        // Set qubits in superposition.
        ApplyToEachA(H, register);

        // Measure all qubits and return.
        return ForEach(MResetZ, register);
    }
}

SampleRandomNumber

You can also write your Q# code in .qs files which are in the same folder as the .ipynb notebook. Running `%workspace reload` will recompile all such .qs files and report the list of available Q# operations and functions. In this case, it finds an operation called `SampleRandomNumber` in a namespace called `Microsoft.Quantum.AzureSamples`.

In [2]:
%workspace reload

Reloading workspace: done!

Microsoft.Quantum.Samples.SampleRandomNumber

Using `%simulate`, you can invoke the built-in Q# functionality to simulate a quantum operation locally and return the result. You can specify any operation that has been defined in the notebook or that has been imported from a .qs file.

In [3]:
%simulate SampleRandomNumber nQubits=3

Zero, One, One

## Executing Q# operations in Azure Quantum

First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in Azure Portal.

Next, you can run `%azure.connect` to connect to the workspace and display the list of provisioned targets that support execution of Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace.

In [4]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

Authenticated using Azure.Identity.AzureCliCredential


Connected to Azure Quantum workspace chgranad-0-15-2103-test5 in location eastus.


Target ID,Current Availability,Average Queue Time (Seconds)
honeywell.hqs-lt-s1,Unavailable,0
honeywell.hqs-lt-s1-apival,Available,0
honeywell.hqs-lt-s2,Degraded,0
honeywell.hqs-lt-s2-apival,Available,0
honeywell.hqs-lt-s1-sim,Available,930
ionq.qpu,Available,1108
ionq.simulator,Available,0


Now, use `%azure.target` to specify the target you'd like to use for job submission.

In [5]:
%azure.target ionq.simulator

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.simulator,Available,0


To submit a job, use `%azure.submit` along with the Q# operation name and any parameters required by that operation. The `%azure.submit` command will return immediately after the job is created. Alternatively, you can use `%azure.execute`, which will submit the job and wait for it to complete.

In [6]:
%azure.submit SampleRandomNumber nQubits=3

Submitting SampleRandomNumber to target ionq.simulator...
Job successfully submitted for 500 shots.
   Job name: SampleRandomNumber
   Job ID: 2737b0fe-883c-4e08-964a-14c22cf6e591


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
SampleRandomNumber,2737b0fe-883c-4e08-964a-14c22cf6e591,Waiting,ionq.simulator,9/28/2021 4:58:17 AM +00:00,,


Running `%azure.status` will display the status of the most recently submitted job in this session. If you want to check the status of a different job, provide the job ID to `%azure.status`.

In [7]:
%azure.status

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
SampleRandomNumber,2737b0fe-883c-4e08-964a-14c22cf6e591,Succeeded,ionq.simulator,9/28/2021 4:58:17 AM +00:00,9/28/2021 4:58:19 AM +00:00,9/28/2021 4:58:19 AM +00:00


Once the job has completed, use `%azure.output` to display the result. Again, you can provide a job ID to `%azure.output` if you want to display the status of a specific job.

In [8]:
%azure.output

Result,Frequency,Histogram
"[0,0,0]",0.125,
"[1,0,0]",0.125,
"[0,1,0]",0.125,
"[1,1,0]",0.125,
"[0,0,1]",0.125,
"[1,0,1]",0.125,
"[0,1,1]",0.125,
"[1,1,1]",0.125,


You can also view the status of all jobs by using `%azure.jobs`. Providing a parameter to this command will filter to just the jobs containing that string. For example, you can query for the status of all jobs named `Microsoft.Quantum.AzureSamples.SampleRandomNumber`.

In [ ]:
%azure.jobs SampleRandomNumber

## Usage information

To list all of the available magic commands, run `%lsmagic`.

In [ ]:
%lsmagic

And to get detailed usage information on any of these magic commands, simply append `?` to its name.

In [11]:
%azure.execute?

%azure.execute:
Submits a job to an Azure Quantum workspace and waits for completion.